In [5]:
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc, recall_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import  OrdinalEncoder
import sqlite3
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
from src import paths
from src import sql_querys

In [6]:
conn = sqlite3.connect(paths.db_path)
cursor = conn.cursor()

In [7]:
query = sql_querys.query_age_model
df = pd.read_sql_query(query, conn)
df.head()

,client_id,age,job,marital,education,gender,client_id,has_deposits,loan,has_insurance,has_mortgage,client_id,mean_balance,max_balance,min_balance,dif_balance,currency
0,249789938,38.0,services,married,secondary,M,249789938,yes,no,n,no,249789938,2669.532734,2775.615374,2515.500059,260.115315,CZK
1,1222646323,46.0,services,divorced,unknown,M,1222646323,no,no,n,no,1222646323,960.562073,1054.517907,886.126906,168.391001,CZK
2,451375919,33.0,admin.,single,secondary,F,451375919,no,no,n,yes,451375919,1221.016419,1303.494818,1100.917203,202.577615,CZK
3,338972671,44.0,self-employed,married,secondary,F,338972671,no,no,y,yes,338972671,297.993265,446.676191,202.053088,244.623103,CZK
4,1472834688,36.0,blue-collar,married,primary,M,1472834688,yes,no,n,yes,1472834688,1919.318145,2011.939205,1853.387429,158.551776,CZK


In [35]:
df = df.T.drop_duplicates().T
df.mean_balance = pd.to_numeric(df.mean_balance)
df.max_balance = pd.to_numeric(df.max_balance)
df.min_balance = pd.to_numeric(df.min_balance)

In [36]:
rate = {'CZK': 1, 'USD': 23, 'EUR': 25}

df.mean_balance = df.apply(lambda row: row['mean_balance'] * rate[row['currency']], axis=1)
df.max_balance = df.apply(lambda row: row['max_balance'] * rate[row['currency']], axis=1)
df.min_balance = df.apply(lambda row: row['min_balance'] * rate[row['currency']], axis=1)

In [37]:
df1 = df.drop(columns='client_id')
df1.dropna(subset=['age'], inplace=True, how='any', axis=0)
y = df1.age
X = df1.drop(columns = 'age')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.preprocessing import LabelEncoder
education_map = {'primary':1, 'secondary':2, 'tertiary':3,'unknown':0}
df.education = df.education.replace(education_map)

label_encoders = {}
for col in ['marital', 'has_deposits', 'loan', "has_insurance", 'has_mortgage', 'gender']:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

C:\Users\spiri\AppData\Local\Temp\ipykernel_3664\1231580676.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.education = df.education.replace(education_map)


In [17]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
ohe_result = ohe.fit_transform(df[['job', 'currency']])
ohe_df = pd.DataFrame(ohe_result.toarray(), columns=ohe.get_feature_names_out(['job', 'currency']))
df = pd.concat([df, ohe_df], axis=1)
df.drop(columns=['job', 'currency'], inplace=True)

In [18]:
df1 = df.drop(columns='client_id')
df1.dropna(subset=['age'], inplace=True, how='any', axis=0)
y = df1.age
X = df1.drop(columns = 'age')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Here we present a pipeline for modeling age, specifically designed to handle missing values within the dataset's age column.

In [41]:

num_features = ['mean_balance', 'max_balance', 'min_balance', 'dif_balance']
cat_features = ['job', 'currency']
labeled_features = ['marital', 'education', 'gender', 'has_deposits', 'loan', 'has_insurance', 'has_mortgage']


# Create transformers for the numerical and categorical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

labeled_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='no')),
    ('label', OrdinalEncoder())])


# Create a column transformer to apply the transformations to the respective columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features),
        ('label', labeled_transformer, labeled_features)
            ])

# Full pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingRegressor(learning_rate=0.05, loss='huber', max_depth=8, 
                max_features='sqrt', min_samples_leaf=10, min_samples_split=10, n_estimators=129))])

In [42]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['mean_balance',
                                                   'max_balance', 'min_balance',
                                                   'dif_balance']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='unknown',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknow...
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='no',
                                                                                 strategy='constant')),
                                                                  ('label',
                                                                   OrdinalEncoder())]),
                                                  ['marital', 'education',
                                                   'gender', 'has_deposits',
                                                   'loan', 'has_insurance',
                                                   'has_mortgage'])])),
                ('classifier',
                 GradientBoostingRegressor(learning_rate=0.05, loss='huber',
                                           max_depth=8, max_features='sqrt',
                                           min_samples_leaf=10,
                                           min_samples_split=10,
                                           n_estimators=129))])

In [43]:
from sklearn.metrics import mean_absolute_error
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

# Print the MAE
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 6.567268109689858


In [44]:
import joblib
joblib.dump(pipeline, 'age_pipeline.pkl')

['age_pipeline.pkl']